# Importing Necessary Libraries

In [259]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from numpy import expand_dims
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Activation,Dropout

# Loading Dataset

In [260]:
df = pd.read_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\FYP-I\\Master\\Data Collection\\CollectedData\\Final_Data (2022).csv", index_col = False)

In [261]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [262]:
df.head(7)

,Province,Country_Region,Date,Day,Month,Year,Lat,Long_,Confirmed,Deaths,...,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax,Temp,Wind,Barometer,Visibility,Weather
0,Balochistan,Pakistan,2020-03-10,10,3,2020,28.328492,65.898403,1.0,0.0,...,0.0,0.0,25.615120,44.61881,10.791250,46.29,15.00,31.43,0.00,Haze.
1,Islamabad,Pakistan,2020-03-10,10,3,2020,33.665087,73.121219,2.0,0.0,...,0.0,0.0,0.004133,0.00000,0.000000,65.14,9.57,34.71,21.33,Clear.
2,Punjab,Pakistan,2020-03-10,10,3,2020,30.811346,72.139132,0.0,0.0,...,0.0,0.0,40.904370,89.53674,11.384460,56.40,2.44,23.67,11.88,Clear.
3,Sindh,Pakistan,2020-03-10,10,3,2020,26.009446,68.776807,0.0,0.0,...,0.0,0.0,22.339800,81.02134,6.179772,65.14,9.57,34.71,21.33,Clear.
4,Khyber Pakhtunkhwa,Pakistan,2020-03-10,10,3,2020,34.485332,72.091690,0.0,0.0,...,0.0,0.0,17.376130,39.58196,5.953569,48.80,4.64,49.20,23.21,Fog.
5,Azad Jammu and Kashmir,Pakistan,2020-03-10,10,3,2020,34.027401,73.947253,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,39.86,1.86,42.43,0.00,Clear.
6,Gilgit-Baltistan,Pakistan,2020-03-10,10,3,2020,35.792146,74.982138,2.0,0.0,...,0.0,0.0,12.495310,23.41426,3.297438,32.71,3.71,23.57,12.82,Fog.


# Creating Tensors for Confirmed Cases

### Generating "X" from Tabular Data

In [263]:
# Getting Daily_Confirmed cases for each province
sConfirmed = df[df['Province'] == 'Sindh']['Daily_Confirmed'].values
bConfirmed = df[df['Province'] == 'Balochistan']['Daily_Confirmed'].values
gConfirmed = df[df['Province'] == 'Gilgit-Baltistan']['Daily_Confirmed'].values
iConfirmed = df[df['Province'] == 'Islamabad']['Daily_Confirmed'].values
pConfirmed = df[df['Province'] == 'Punjab']['Daily_Confirmed'].values
kConfirmed = df[df['Province'] == 'Khyber Pakhtunkhwa']['Daily_Confirmed'].values
aConfirmed = df[df['Province'] == 'Azad Jammu and Kashmir']['Daily_Confirmed'].values

In [264]:
len(sConfirmed) == len(bConfirmed) == len(iConfirmed) == len(pConfirmed) == len(kConfirmed) == len(aConfirmed) == len(gConfirmed)

True

In [265]:
len(bConfirmed)

703

In [266]:
# Combining Daily_Confirmed cases to make 4*4 Matrix
sbConfirmed = np.zeros(len(sConfirmed)*2)
giConfirmed = np.zeros(len(sConfirmed)*2)
pkConfirmed = np.zeros(len(sConfirmed)*2)
axConfirmed = np.zeros(len(sConfirmed)*2)

In [267]:
len(sbConfirmed)
# S B
# G I
# P K
# A -

1406

In [268]:
for i in range(len(sbConfirmed)):
    if(i%2 == 0):
        sbConfirmed[i] = sConfirmed[i//2]
    elif(i%2 == 1):
        sbConfirmed[i] = bConfirmed[(i - 1)//2]

In [269]:
for i in range(len(giConfirmed)):
    if(i%2 == 0):
        giConfirmed[i] = gConfirmed[i//2]
    elif(i%2 == 1):
        giConfirmed[i] = iConfirmed[(i - 1)//2]

In [270]:
for i in range(len(pkConfirmed)):
    if(i%2 == 0):
        pkConfirmed[i] = pConfirmed[i//2]
    elif(i%2 == 1):
        pkConfirmed[i] = kConfirmed[(i - 1)//2]

In [271]:
for i in range(len(axConfirmed)):
    if(i%2 == 0):
        axConfirmed[i] = aConfirmed[i//2]
    elif(i%2 == 1):
        axConfirmed[i] = -1

In [272]:
AllConfirmed = np.zeros((4, len(sbConfirmed)))

In [273]:
AllConfirmed[0] = sbConfirmed
AllConfirmed[1] = giConfirmed
AllConfirmed[2] = pkConfirmed
AllConfirmed[3] = axConfirmed

In [274]:
tempArr = np.zeros((4, 8))

In [275]:
AllConfirmed.shape[1]/8

175.75

In [276]:
tensorsArr = np.zeros((175, 4, 8))

In [277]:
stIndex = 0
enIndex = 8
for tIndex in range(175):
    for i in range(4):
        tempArr[i] = AllConfirmed[i][stIndex:enIndex]
    stIndex += 8
    enIndex += 8
    tensorsArr[tIndex] = tempArr

In [278]:
len(tensorsArr)

175

In [279]:
finalTensorsArr = np.zeros((88, 8, 8))

In [280]:
stIndex = 0
enIndex = 8
for tIndex in range(0, 176, 2):
    tfIndex = tIndex//2
    for i in range(8):
        if(i < 4):
#             print(tensors[tIndex][i])
            finalTensorsArr[tfIndex][i] = tensorsArr[tIndex][i]
#             print(FinalTensors[tfIndex][i])
        else:
#             print(tensors[tIndex][i-4])
            finalTensorsArr[tfIndex][i] = tensorsArr[tIndex][i-4]
#             print(FinalTensors[tfIndex][i-4])
#     print(FinalTensors[tfIndex])
#     print("\n")

In [324]:
finalTensorsArr.shape

(88, 8, 8)

In [325]:
finalTensorsArr[0]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  5.],
       [ 2.,  2.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0., -1.,  0., -1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  5.],
       [ 2.,  2.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0., -1.,  0., -1.]])

In [326]:
88 * 8

704

In [282]:
confirmedTensors = []

In [283]:
for i in range(88):
    confirmedTensors.append(tf.convert_to_tensor(finalTensorsArr[i], dtype=tf.float64))

In [284]:
confirmedTensors[87]

<tf.Tensor: shape=(8, 8), dtype=float64, numpy=
array([[ 1.779e+03,  7.700e+01,  1.952e+03,  7.700e+01,  1.527e+03,
         7.400e+01,  8.630e+02,  3.400e+01],
       [ 6.900e+01,  6.150e+02,  3.600e+01,  4.990e+02,  1.800e+01,
         4.650e+02,  5.000e+00,  2.910e+02],
       [ 2.031e+03,  1.258e+03,  1.597e+03,  1.515e+03,  1.196e+03,
         1.480e+03,  1.052e+03,  9.200e+02],
       [ 5.710e+02, -1.000e+00,  4.610e+02, -1.000e+00,  2.130e+02,
        -1.000e+00,  1.730e+02, -1.000e+00],
       [ 1.779e+03,  7.700e+01,  1.952e+03,  7.700e+01,  1.527e+03,
         7.400e+01,  8.630e+02,  3.400e+01],
       [ 6.900e+01,  6.150e+02,  3.600e+01,  4.990e+02,  1.800e+01,
         4.650e+02,  5.000e+00,  2.910e+02],
       [ 2.031e+03,  1.258e+03,  1.597e+03,  1.515e+03,  1.196e+03,
         1.480e+03,  1.052e+03,  9.200e+02],
       [ 5.710e+02, -1.000e+00,  4.610e+02, -1.000e+00,  2.130e+02,
        -1.000e+00,  1.730e+02, -1.000e+00]])>

In [285]:
len(confirmedTensors)

88

In [286]:
len(confirmedTensors) * 8

704

### Generating "y" from Tabular Data

In [336]:
y_tempConfirmed = []

In [337]:
for i in range(0, len(df['Date']), 7):
    try:
        y_tempConfirmed.append(df[df['Date'] == str(datetime.datetime.strptime(df['Date'].iloc[i], '%Y-%m-%d') + datetime.timedelta(days=7))[:10]]['Daily_Confirmed'].values)
#         for data in df[(df['Date'] == str(datetime.datetime.strptime(df['Date'].iloc[i], '%Y-%m-%d') + datetime.timedelta(days=8))[:10])]['Daily_Confirmed']:
#             print(data)
#         print(df[
#         (df['Date'] == str(datetime.datetime.strptime(df['Date'].iloc[i], '%Y-%m-%d') + datetime.timedelta(days=8))[:10])
#     & (df['Province'] == 'Sindh')]['Daily_Confirmed'].values[0])
    except:
        print(i)
        continue

In [338]:
y_tempConfirmed[0]

array([ 1.,  1., 25.,  0.,  2.,  0.,  0.])

In [344]:
y_tempConfirmed = y_tempConfirmed[:696]

In [292]:
# Balochistan
# Islamabad
# Punjab
# Sindh
# Khyber Pakhtunkhwa
# Azad Jammu and Kashmir
# Gilgit-Baltistan

In [293]:
bCases = []
iCases = []
pCases = []
sCases = []
kCases = []
aCases = []
gCases = []

In [294]:
for i in range(len(y_tempConfirmed)):
    try:
        bCases.append(y_tempConfirmed[i][0])
        iCases.append(y_tempConfirmed[i][1])
        pCases.append(y_tempConfirmed[i][2])
        sCases.append(y_tempConfirmed[i][3])
        kCases.append(y_tempConfirmed[i][4])
        aCases.append(y_tempConfirmed[i][5])
        gCases.append(y_tempConfirmed[i][6])
    except:
        continue

In [295]:
bSumCases = np.zeros(100)
iSumCases = np.zeros(100)
pSumCases = np.zeros(100)
sSumCases = np.zeros(100)
kSumCases = np.zeros(100)
aSumCases = np.zeros(100)
gSumCases = np.zeros(100)

In [352]:
for i in range(0, len(bCases), 7): 
    print(i//7)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [358]:
for i in range(0, len(bCases) - 7, 7): 
    try:
        for j in range(7):
            if (i > 0):
#                 print(int(i//7))
                bSumCases[int(i/7)] += bCases[i+j]
                iSumCases[int(i/7)] += iCases[i+j]
                pSumCases[int(i/7)] += pCases[i+j]
                sSumCases[int(i/7)] += sCases[i+j]
                kSumCases[int(i/7)] += kCases[i+j]
                aSumCases[int(i/7)] += aCases[i+j]
                gSumCases[int(i/7)] += gCases[i+j]
            else:
#                 print(int(i))
                bSumCases[int(i)] += bCases[i+j]
                iSumCases[int(i)] += iCases[i+j]
                pSumCases[int(i)] += pCases[i+j]
                sSumCases[int(i)] += sCases[i+j]
                kSumCases[int(i)] += kCases[i+j]
                aSumCases[int(i)] += aCases[i+j]
                gSumCases[int(i)] += gCases[i+j]
    except:
        continue

In [359]:
ySumConfirmed = np.zeros((100, 7, 1))

In [360]:
ySumConfirmed[0]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [361]:
for i in range(100):
    ySumConfirmed[i][0] = bSumCases[i] 
    ySumConfirmed[i][1] = iSumCases[i]  
    ySumConfirmed[i][2] = pSumCases[i] 
    ySumConfirmed[i][3] = sSumCases[i]  
    ySumConfirmed[i][4] = kSumCases[i] 
    ySumConfirmed[i][5] = aSumCases[i] 
    ySumConfirmed[i][6] = gSumCases[i] 

In [364]:
ySumConfirmed[99]

array([[ 1242.],
       [ 6948.],
       [25506.],
       [24552.],
       [33921.],
       [ 5409.],
       [ 1098.]])

In [365]:
ySumConfirmed[99]

array([[ 1242.],
       [ 6948.],
       [25506.],
       [24552.],
       [33921.],
       [ 5409.],
       [ 1098.]])

In [366]:
yConfirmedTensors = np.zeros((100, 7, 1))

In [367]:
for i in range(100):
    yConfirmedTensors[i] = tf.convert_to_tensor(ySumConfirmed[i], dtype=tf.float64)

In [369]:
yConfirmedTensors[99]

array([[ 1242.],
       [ 6948.],
       [25506.],
       [24552.],
       [33921.],
       [ 5409.],
       [ 1098.]])

# Data Preparation

In [304]:
len(confirmedTensors)

88

In [305]:
len(yConfirmedTensors[:88])

88

In [306]:
X = confirmedTensors
y = yConfirmedTensors[:88]

In [307]:
X = np.array(X)

In [308]:
X.shape

(88, 8, 8)

In [309]:
# Expanding the dimensions for the CNN
X = expand_dims(X, axis=3)

In [310]:
X.shape

(88, 8, 8, 1)

In [311]:
y.shape

(88, 7, 1)

# Model Implementation for Predicting Confirmed Cases for each Province in next 7 days

In [312]:
model = Sequential()

model.add(Conv2D(8, (2, 2), input_shape=(8, 8, 1)))
model.add(Activation("relu"))
model.add(MaxPooling2D((2,2), padding='same'))
# model.add(Dropout(0.25))

model.add(Conv2D(4, (2, 2)))
model.add(Activation("relu"))
model.add(MaxPooling2D((2,2), padding='same'))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(36, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(7, activation='linear'))

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam')

In [313]:
88 * 0.6

52.8

In [314]:
X_train = X[:52]
y_train = y[:52]

In [315]:
X_test = X[52:]
y_test = y[52:]

In [316]:
model.fit(X_train, y_train, batch_size = 4, epochs = 50, validation_split = 0.2)

Epoch 1/50
11/11 [==============================] - 1s 31ms/step - loss: 10527961.0000 - val_loss: 7426702.0000
Epoch 2/50
11/11 [==============================] - 0s 4ms/step - loss: 9620948.0000 - val_loss: 7194618.0000
Epoch 3/50
11/11 [==============================] - 0s 4ms/step - loss: 8546802.0000 - val_loss: 7325736.5000
Epoch 4/50
11/11 [==============================] - 0s 4ms/step - loss: 7506032.5000 - val_loss: 7791112.0000
Epoch 5/50
11/11 [==============================] - 0s 4ms/step - loss: 6180360.5000 - val_loss: 8953580.0000
Epoch 6/50
11/11 [==============================] - 0s 4ms/step - loss: 4989020.5000 - val_loss: 11259412.0000
Epoch 7/50
11/11 [==============================] - 0s 4ms/step - loss: 3721488.7500 - val_loss: 14078959.0000
Epoch 8/50
11/11 [==============================] - 0s 4ms/step - loss: 2938243.5000 - val_loss: 17409090.0000
Epoch 9/50
11/11 [==============================] - 0s 4ms/step - loss: 2237739.0000 - val_loss: 20388084.0000
Epoc

In [321]:
prediction = model.predict(X_test)

In [322]:
prediction[-1]

array([ 1217.508  ,  2060.8367 , 10392.71   , 13156.645  ,  4059.9255 ,
         231.00931,    64.74897], dtype=float32)

In [320]:
y_test[-1]

array([[  44.],
       [ 144.],
       [ 576.],
       [1044.],
       [ 482.],
       [  24.],
       [   5.]])